In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split





#可以用決策數分析來協助報告

In [135]:
ls = pd.read_csv("train/train.csv",encoding="BIG5")

C:\Users\a6504\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (17,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [136]:
check=ls.isna()
column =ls.columns.values.tolist()
for i in column:
    if sum(check[i]) >500:  #將缺失值超過500個的column刪除
        #20000:74
        #15000"73
        #10000:73
        #5000:72
        #2000:71
        #500:70
        #100:67
        #1:59
        ls=ls.drop([i], axis=1)
ls=ls.fillna(0) #將其餘缺失值以0補上

ls=ls.replace("低",0)#將可量化的值取代
ls=ls.replace("中",1)
ls=ls.replace("中高",2)
ls=ls.replace("高",3)

ls = pd.get_dummies(ls)# 將剩餘不可量化、單純為類別的變數轉換為1000、0100的格式
ls=ls.drop("Y1_N",axis=1)#上一步驟的轉換會多轉一個沒有意義的column 將這個刪掉
ls=ls.drop("CUS_ID",axis=1)#流水號刪除
b = ls.columns.values.tolist()#b=所有column的名子的串列
b.remove("Y1_Y")#將答案抹除
repet = ls[ls["Y1_Y"]==1]
#repet =  pd.concat([repet,repet,repet,repet,repet,repet,repet,repet,repet,repet])#將有續約的複製10000筆，並以10000筆為一個單位
ls = pd.concat([ls,repet,repet])#將有續約的複製100000筆以增加權重，解決比例懸殊問題

In [128]:
ls

,AGE,L1YR_A_ISSUE_CNT,L1YR_B_ISSUE_CNT,CHANNEL_A_POL_CNT,CHANNEL_B_POL_CNT,APC_CNT,INSD_CNT,INSD_1ST_AGE,LIFE_CNT,AG_CNT,...,IF_ADD_INSD_IND_Y,CUST_9_SEGMENTS_CD_A,CUST_9_SEGMENTS_CD_B,CUST_9_SEGMENTS_CD_C,CUST_9_SEGMENTS_CD_D,CUST_9_SEGMENTS_CD_E,CUST_9_SEGMENTS_CD_F,CUST_9_SEGMENTS_CD_G,CUST_9_SEGMENTS_CD_H,Y1_Y
0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,1,...,0,1,0,0,0,0,0,0,0,0
2,0,1,0,1,0,1,0,1,1,1,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,1,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,1,0,1,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,1,2,...,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
#以上為資料預處理

In [4]:
#以下為資料分析

In [137]:
#分割資料為測試集
all_inputs = ls[b].values
all_classes = ls['Y1_Y'].values
(train_inputs, test_inputs, train_classes, test_classes) = train_test_split(all_inputs, all_classes, train_size=0.7, random_state=1)

C:\Users\a6504\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [41]:
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn.kernel_approximation import RBFSampler#調整參數用
from sklearn.naive_bayes import GaussianNB

In [138]:

dtc = LinearSVC(intercept_scaling=1,class_weight="balanced")
#dtc = linear_model.SGDClassifier()
#dtc = GaussianNB()
#dtc = BaggingClassifier()
dtc.fit(train_inputs, train_classes)
dtc.score(test_inputs, test_classes)

C:\Users\a6504\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.8433012820512821

In [139]:
dtc.coef_

array([[-1.52307969e-01,  2.85930981e-02, -1.88927227e-03,
        -3.21226677e-02,  6.37442309e-02, -1.60196998e-02,
        -5.29445915e-02,  5.25236976e-02, -9.95319323e-04,
        -9.26908175e-02,  1.68172607e-01, -1.62547977e-03,
         2.66134759e-01,  1.52138713e+00,  2.46365066e-03,
         5.06586555e-03,  1.44870707e+00,  4.13912503e-02,
         5.64549448e-02,  2.94453471e-02,  2.83123431e-02,
        -1.33155583e-01, -8.55691160e-03, -4.50601792e-02,
        -1.07009844e-01,  1.96219627e-01, -4.21943708e-02,
        -3.19053295e-02, -3.82291043e-02,  7.88221238e-03,
         4.41987736e-02, -1.92262855e-02,  6.24083706e-02,
         3.37154783e-02, -1.42779929e-01,  1.59429673e-01,
        -1.67112956e-01,  1.83762701e-01, -1.49334004e-02,
         3.15831451e-02, -3.11580799e-02,  4.78078246e-02,
         2.07124099e-01, -1.90474354e-01,  4.64871439e-02,
        -2.98373992e-02,  3.54821928e-02, -1.88324481e-02,
         3.14709289e-02, -1.48211842e-02,  1.96104774e-0

In [140]:
dtc.intercept_

array([0.01664974])

In [141]:
c = dtc.predict(test_inputs)
sum(c)
#sum(test_classes)

4810

In [142]:
print("總預測數 %d "%len(c))
print("預測有續約%d "%sum(test_classes == 1))
print("預測無續約 %d "%sum(test_classes == 0))
print("有續約且預測為有續約數:%d"%sum((c == test_classes)&(test_classes == 1)))
print("無續約且預測為無續約數:%d"%sum((c == test_classes)&(test_classes == 0)))
print("有續約但預測為無續約數:%d"%sum((c != test_classes)&(test_classes == 1)))
print("無續約但預測為有續約數:%d"%sum((c != test_classes)&(test_classes == 0)))
print("有續約且預測為有續約率:%f"%(sum((c == test_classes)&(test_classes == 1))/sum(test_classes == 1)))
print("無續約且預測為無續約率:%f"%(sum((c == test_classes)&(test_classes == 0))/sum(test_classes == 0)))
print("有續約但預測為無續約率:%f"%(sum((c != test_classes)&(test_classes == 1))/sum(test_classes == 1)))
print("無續約但預測為有續約率:%f"%(sum((c != test_classes)&(test_classes == 0))/sum(test_classes == 0)))
print("正確率%f"%(sum(c == test_classes)/len(test_classes)))

總預測數 31200 
預測有續約1797 
預測無續約 29403 
有續約且預測為有續約數:859
無續約且預測為無續約數:25452
有續約但預測為無續約數:938
無續約但預測為有續約數:3951
有續約且預測為有續約率:0.478019
無續約且預測為無續約率:0.865626
有續約但預測為無續約率:0.521981
無續約但預測為有續約率:0.134374
正確率0.843301


In [ ]:
#輸入值 : ls

#模型  :  dtc = LinearSVC(intercept_scaling=1,class_weight="balanced")

#結果
"""
總預測數 30000 
預測有續約603 
預測無續約 29397 
有續約且預測為有續約數:127
無續約且預測為無續約數:28420
有續約但預測為無續約數:476
無續約但預測為有續約數:977
有續約且預測為有續約率:0.210614
無續約且預測為無續約率:0.966765
有續約但預測為無續約率:0.789386
無續約但預測為有續約率:0.033235
正確率0.951567
"""

In [ ]:
#輸入值 :  ls+複製一次有續約

#模型 :  dtc = LinearSVC(intercept_scaling=1,class_weight="balanced")
#結果
"""
總預測數 30600 
預測有續約1195 
預測無續約 29405 
有續約且預測為有續約數:384
無續約且預測為無續約數:27058
有續約但預測為無續約數:811
無續約但預測為有續約數:2347
有續約且預測為有續約率:0.321339
無續約且預測為無續約率:0.920184
有續約但預測為無續約率:0.678661
無續約但預測為有續約率:0.079816
正確率0.896797
"""

In [ ]:
#輸入值 :  ls+複製兩次有續約

#模型 :  dtc = LinearSVC(intercept_scaling=1,class_weight="balanced")
#結果
"""
總預測數 31200 
預測有續約1797 
預測無續約 29403 
有續約且預測為有續約數:859
無續約且預測為無續約數:25452
有續約但預測為無續約數:938
無續約但預測為有續約數:3951
有續約且預測為有續約率:0.478019
無續約且預測為無續約率:0.865626
有續約但預測為無續約率:0.521981
無續約但預測為有續約率:0.134374
正確率0.843301
"""